In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from pycaret import regression
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import metrics
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical, Real
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import os, copy, random, pickle, seaborn, datetime, shap

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# SoC classification

In [ ]:
data = pd.read_csv('./SoC data.csv')

mms_classification = MinMaxScaler()
mms_classification.fit(data[col])
data_transform = pd.DataFrame(mms_classification.transform(data[col]), columns=col)
data_transform['soc_class'] = data['soc_class']

corr = list(data_transform.corr()['soc'].sort_values(key=abs, ascending=False).keys())[1:]
temp_corr = data_transform.corr()
temp_corr = temp_corr.loc[corr, corr]

t_feature_col = copy.deepcopy(col[1:])
for i in t_feature_col:
    if (np.isnan(data_transform.corr().loc[i, 'soc'])):
        t_feature_col.remove(i)

feature_col = copy.deepcopy(t_feature_col)
for i in range(len(t_feature_col)):
    t = []
    for j in range(i+1, len(feature_col)):
        if (abs(temp_corr.loc[feature_col[i], feature_col[j]]) > 0.75):
            t.append(feature_col1[j])
    for j in t:
        feature_col.remove(j)

feature = ['soc_class']+feature_col
data_transform = data_transform[feature]

opt_condition = LGBMClassifier(verbose=-1, learning_rate=0.19763117861267274, n_estimators=101, max_depth=3, min_child_samples=100, min_child_weight=1.1424465350768398, num_leaves=150, colsample_bytree=0.6634449816853505, subsample=0.8639560911474804, reg_alpha=0.3345612399250176, reg_lambda=1.955106417669215)
opt_condition.fit(data_transform[feature_col], data_transform['soc_class'])


In [ ]:
test_size = 0.15

temp_X_train, temp_X_test, temp_y_train, temp_y_test = train_test_split(soc_data, soc_data['100_OCV'], test_size=test_size)
feature_col = ['OCV', 'dVi', 'dV', 'skew']

train = pd.DataFrame()
c = 0
for i in temp_X_train.index:
    for soc in range(0, 100+1, 5):
        if soc >= 80:
            train.loc[c, 'soc_class'] = 2
        elif soc >= 10:
            train.loc[c, 'soc_class'] = 1
        elif soc >= 0:
            train.loc[c, 'soc_class'] = 0

        train.loc[c, 'soc'] = soc
        for k in col[1:]:
            train.loc[c, k] = temp_X_train.loc[i, '{}_{}'.format(soc, k)]

        c += 1

test = pd.DataFrame()
c = 0
for i in temp_X_test.index:
    for soc in range(0, 100+1, 5):
        if soc >= 80:
            test.loc[c, 'soc_class'] = 2
        elif soc >= 10:
            test.loc[c, 'soc_class'] = 1
        elif soc >= 0:
            test.loc[c, 'soc_class'] = 0

        test.loc[c, 'soc'] = soc
        for k in col[1:]:
            test.loc[c, k] = temp_X_test.loc[i, '{}_{}'.format(soc, k)]

        c += 1

mms = MinMaxScaler()
mms.fit(train[col])
train_data_transform = pd.DataFrame(mms.transform(train[col]), columns=col)
train_data_transform['soc_class'] = train['soc_class']

test_data_transform = pd.DataFrame(mms.transform(test[col]), columns=col)
test_data_transform['soc_class'] = test['soc_class']

print(feature_col, len(train_data_transform[feature_col]), len(test_data_transform[feature_col]))
condition_model = LGBMClassifier(verbose=-1).fit(train_data_transform[feature_col], train_data_transform['soc_class'])

train_score = metrics.accuracy_score(train_data_transform['soc_class'], condition_model.predict(train_data_transform[feature_col]))
test_score = metrics.accuracy_score(test_data_transform['soc_class'], condition_model.predict(test_data_transform[feature_col]))
print(train_score, test_score)


opt_condition = LGBMClassifier(verbose=-1, learning_rate=0.19763117861267274, n_estimators=101, max_depth=3, min_child_samples=100, min_child_weight=1.1424465350768398, num_leaves=150, colsample_bytree=0.6634449816853505, subsample=0.8639560911474804, reg_alpha=0.3345612399250176, reg_lambda=1.955106417669215)
opt_condition.fit(data_transform[feature_col], data_transform['soc_class'])

train_score = metrics.accuracy_score(train_data_transform['soc_class'], opt_condition.predict(train_data_transform[feature_col]))
test_score = metrics.accuracy_score(test_data_transform['soc_class'], opt_condition.predict(test_data_transform[feature_col]))
print(train_score, test_score)

plt.scatter(train_data_transform['soc'], opt_condition.predict(train_data_transform[feature_col]), alpha=0.5)
plt.scatter(test_data_transform['soc'], opt_condition.predict(test_data_transform[feature_col]), alpha=0.5)
plt.show()

In [ ]:
feature_col = ['OCV', 'dVi', 'dV', 'skew']

opt_condition = LGBMClassifier(verbose=-1, learning_rate=0.19763117861267274, n_estimators=101, max_depth=3, min_child_samples=100, min_child_weight=1.1424465350768398, num_leaves=150, colsample_bytree=0.6634449816853505, subsample=0.8639560911474804, reg_alpha=0.3345612399250176, reg_lambda=1.955106417669215)
opt_condition.fit(data_transform[feature_col], data_transform['soc_class'])

tree_explainer = shap.TreeExplainer(opt_condition)
shap_values = tree_explainer.shap_values(data_transform[feature_col], data_transform['soc_class'])

for class_idx in range(3):
    print(f"Class {class_idx} summary plot:")
    shap.summary_plot(shap_values[:, :, class_idx], data_transform[feature_col], show=True)

# SoH regression

In [ ]:
category = {0 : [['char', 'dischar'], ['pulse', 'rest'], [100, 95, 90, 85, 80], [float(x) for x in [1.65, 2]]], 
            1 : [['char', 'dischar'], ['pulse', 'rest'], [75, 70, 65, 60], [float(x) for x in [1.65, 2]]],
            2 : [['char', 'dischar'], ['pulse', 'rest'], [55, 50, 45, 40, 35, 30, 25, 20, 15, 10], [float(x) for x in [1.65, 2]]],
            3 : [['char', 'dischar'], ['pulse', 'rest'], [5, 0], [float(x) for x in [1.65, 2]]]}

file_list = []
path_1 = './SoH_protocol/'
for file_0 in os.listdir(path_1):
    for file_1 in os.listdir(path_1+file_0):
        file_list.append('{}{}/{}'.format(path_1, file_0, file_1))

soh = pd.read_csv('./SoH.csv', index_col=0)
resistance = pd.read_csv('./SoH_resistance.csv', index_col=0)
ir_drop = pd.read_csv('./SoH_ir_drop.csv', index_col=0)

cycle = soh.columns
with pd.ExcelWriter('./SoH_features.xlsx') as writer:
    for e in range(4):
        for a in category[e][2]:
            temp = pd.DataFrame()
            for file in soh.index:
                for cyc in cycle:
                    temp_path='{}{}/'.format(path_1, cyc)
                    if '{}_{}.csv'.format(file, cyc) in os.listdir(temp_path):
                        DCPR = pd.read_csv('{}{}.csv'.format(temp_path, file))
                        for x, c in enumerate(category[e][3]):
                            for d in category[e][1]:
                                for b in category[e][0]:    
                                    temp_volt = []
                                    for z, y in enumerate(DCPR['{}_{}C_{}_{}'.format(a, c, b, d)]):
                                        temp_volt.append(y)
                                        if (y > 4.2) or (y < 2.5):
                                            if DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+1] == DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+1]:
                                                temp_volt.append(DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+1])
                                            if DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+2] == DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+2]:
                                                temp_volt.append(DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+2])
                                            if DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+3] == DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+3]:
                                                temp_volt.append(DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+3])
                                            break
                                        elif y != y:
                                            del temp_volt[-1]
                                            break

                                    temp.loc['{}_{}'.format(file, cyc), 'SoH'] = soh.loc[file, cyc]
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_{}_kurt'.format(c, b, d)] = pd.DataFrame(temp_volt)[0].kurt()
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_{}_skew'.format(c, b, d)] = pd.DataFrame(temp_volt)[0].skew()
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_{}_initial'.format(c, b, d)] = temp_volt[0]
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_{}_final'.format(c, b, d)] = temp_volt[-1]
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_{}_R'.format(c, b, d)] = resistance.loc[file, '{}_{}_{}C_{}_{}'.format(cyc, a, c, b, d)]
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_{}_IR'.format(c, b, d)] = ir_drop.loc['{}_{}'.format(file, cyc), '{}_{}C_{}_{}'.format(a, c, b, d)]

                                    if b == 'char':
                                        if a >= 10:
                                            temp.loc['{}_{}'.format(file, cyc), '{} C_{}_R_ratio'.format(c, d)] = resistance.loc[file, '{}_{}_{}C_char_{}'.format(cyc, a, c, d)]/resistance.loc[file, '{}_{}_{}C_dischar_{}'.format(cyc, a, c, d)]
                                        else:
                                            temp.loc['{}_{}'.format(file, cyc), '{} C_{}_R_ratio'.format(c, d)] = resistance.loc[file, '{}_{}_{}C_dischar_{}'.format(cyc, a, c, d)]/resistance.loc[file, '{}_{}_{}C_char_{}'.format(cyc, a, c, d)]
            
                                        if d == 'pulse':
                                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_{}_time'.format(c, b, d)] = len(temp_volt)/10
                               
                                if d == 'pulse':
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_hysteresis'.format(c, d)] = (max(DCPR['{}_{}C_char_{}'.format(a, c, d)])-min(DCPR['{}_{}C_dischar_{}'.format(a, c, d)]))/crate_current[str(c)]
      
                            for j in category[e][3][x+1:]:
                                for b in category[e][0]:
                                    if resistance.loc[file, '{}_{}_{}C_{}_{}'.format(cyc, a, c, b, d)] == 0.0:
                                        temp.loc['{}_{}'.format(file, cyc), '{} C_R/{} C_R_{}_{}'.format(j ,c, b, d)] = 0.0
                                    else:
                                        temp.loc['{}_{}'.format(file, cyc), '{} C_R/{} C_R_{}_{}'.format(j ,c, b, d)] = (resistance.loc[file, '{}_{}_{}C_{}_{}'.format(cyc, a, j, b, d)]/resistance.loc[file, '{}_{}_{}C_{}_{}'.format(cyc, a, c, b, d)])                 
            
            temp.to_excel(writer, sheet_name=str(a))

In [ ]:
file_paht = './SoH_features.xlsx'

feature_col = ['2.0 C_char_rest_IR', '1.65 C_dischar_pulse_IR', '2.0 C_pulse_hysteresis', '2.0 C_char_pulse_R', '1.65 C_dischar_pulse_R', '1.65 C_dischar_pulse_skew', '2.0 C_char_rest_skew', '2.0 C_char_pulse_final', '1.65 C_dischar_rest_skew', '2.0 C_char_pulse_skew', '2.0 C_dischar_pulse_final', '1.65 C_rest_R_ratio']

df = pd.DataFrame()
for i in range(0, 100+1, 5):
    temp = pd.read_excel(file_path, sheet_name=str(i), index_col=0)
    df = pd.concat([df, temp])

temp = temp.reset_index(drop=True)
df = df.reset_index(drop=True)
print(len(df))

mms = MinMaxScaler()
mms.fit(df[feature_col])

data_transform = pd.DataFrame(mms.transform(df[feature_col]), columns=feature_col)
data_transform['SoH'] = df['SoH']

ml = RandomForestRegressor(n_jobs=-1, max_depth=18, min_samples_leaf=2, min_samples_split=2, n_estimators=261, bootstrap=False, max_features=0.5)
ml.fit(data_transform[feature_col], data_transform['SoH'])

In [ ]:
tree_explainer = shap.TreeExplainer(ml)
shap_values = tree_explainer.shap_values(data_transform[feature_col])
shap.summary_plot(shap_values, data_transform[feature_col], show=True)

In [ ]:
train_list, test_list = [], []
train_protocol, test_protocol = [], []

for sheet in range(0, 100+1, 5):
    temp = pd.read_excel(file_path, sheet_name=str(sheet))
    
    train_df, test_df = train_test_split(temp, test_size=0.15, shuffle=True)

    train_list.append(train_df)
    test_list.append(test_df)

    if sheet < 10:
        train_protocol += [1]*len(train_df)
        test_protocol += [1]*len(test_df)
    elif sheet < 80:
        train_protocol += [2]*len(train_df)
        test_protocol += [2]*len(test_df)
    else:
        train_protocol += [3]*len(train_df)
        test_protocol += [3]*len(test_df)
        
train = pd.concat(train_list, ignore_index=True)
test = pd.concat(test_list, ignore_index=True)
#################
mms = MinMaxScaler()
mms.fit(df[feature_col])

train_data_transform = pd.DataFrame(mms.transform(train[feature_col]), columns=feature_col)
train_data_transform['SoH'] = list(train['SoH'])

test_data_transform = pd.DataFrame(mms.transform(test[feature_col]), columns=feature_col)
test_data_transform['SoH'] = list(test['SoH'])

ml = RandomForestRegressor(n_jobs=-1, max_depth=18, min_samples_leaf=2, min_samples_split=2, n_estimators=261, bootstrap=False, max_features=0.5)
ml.fit(train_data_transform[feature_col], train_data_transform['SoH'])

train_score = metrics.root_mean_squared_error(train['SoH'], ml.predict(train_data_transform[feature_col]))
test_score = metrics.root_mean_squared_error(test['SoH'], ml.predict(test_data_transform[feature_col]))
print('RMSE', train_score, test_score)

train_score = metrics.r2_score(train['SoH'], ml.predict(train_data_transform[feature_col]))
test_score = metrics.r2_score(test['SoH'], ml.predict(test_data_transform[feature_col]))
print('R2 score', train_score, test_score)
print()

print(f"전체 train shape: {train.shape}", f"전체 test shape: {test.shape}")
plt.scatter(train['SoH'], ml.predict(train_data_transform[feature_col]), alpha=0.5)
plt.scatter(test['SoH'], ml.predict(test_data_transform[feature_col]), alpha=0.5)
plt.show()

# Expandability

## SoC classification

In [ ]:
soc_data = pd.read_csv('./SoC data_expandability.csv')

data = pd.DataFrame()
for i in soc_data.index x)]:
    for soc in range(0, 100+1, 5):
        if soc >= 80:
            data.loc[c, 'soc_class'] = 2
        elif soc >= 10:
            data.loc[c, 'soc_class'] = 1
        elif soc >= 0:
            data.loc[c, 'soc_class'] = 0

        data.loc[c, 'soc'] = soc
        for k in ['OCV', 'Vi', 'Vf', 'dVi', 'dVf', 'dV', 'skew', 'kurt']:
            if k=='dVi':
                if '50E' in i:
                    data.loc[c, k] = soc_data.loc[i, '{}_{}'.format(soc, k)]/0.8/(202/166)+0.012
                elif 'M50' in i:
                    data.loc[c, k] = soc_data.loc[i, '{}_{}'.format(soc, k)]/0.8/(198/166)+0.012
                    
            elif k=='dV':
                if '50E' in i:
                    data.loc[c, k] = soc_data.loc[i, '{}_{}'.format(soc, k)]/0.9/(202/166)+0.002
                elif 'M50' in i:
                    data.loc[c, k] = soc_data.loc[i, '{}_{}'.format(soc, k)]/0.9/(202/166)+0.002

            else:
                data.loc[c, k] = soc_data.loc[i, '{}_{}'.format(soc, k)]
            
        c += 1

data_transform = pd.DataFrame(mms_condition.transform(data[col]), columns=col)
data_transform['soc'] = data['soc']

opt_condition = LGBMClassifier(verbose=-1, learning_rate=0.19763117861267274, n_estimators=101, max_depth=3, min_child_samples=100, min_child_weight=1.1424465350768398, num_leaves=150, colsample_bytree=0.6634449816853505, subsample=0.8639560911474804, reg_alpha=0.3345612399250176, reg_lambda=1.955106417669215)
opt_condition.fit(data_transform[feature_col], data_transform['soc_class'])

plt.plot([10, 10], [-0.5, 7.5], 'k--')
plt.plot([80, 80], [-0.5, 7.5], 'k--')

c = 0
for i, j, k in zip(opt_condition.predict(data_transform[feature_col]), data['soc_class'], data['soc']):
    if (i) != int(j):
        plt.scatter(k, i, alpha=0.7)
        print(k)
        c += 1

plt.xlabel('SoC', fontsize=15)
plt.ylabel('Predicted SoC range', fontsize=15)
plt.xticks(range(0, 100+1, 10), fontsize=13)
plt.yticks(range(0, 4+1), fontsize=13)
plt.ylim([-0.3, 2.3])
plt.show()

## SoH regression

In [ ]:
## inverse
crate_current_50E = {'0.1':4.9, '1.65':8.085, '2.0':9.8}
crate_current_M50LT = {'0.1':4.8, '1.65':7.92, '2.0':9.6}

category = {0 : [['char', 'dischar'], ['pulse', 'rest'], [100, 95, 90, 85, 80], [float(x) for x in [1.65, 2]]], 
            1 : [['char', 'dischar'], ['pulse', 'rest'], [75, 70, 65, 60], [float(x) for x in [1.65, 2]]],
            2 : [['char', 'dischar'], ['pulse', 'rest'], [55, 50, 45, 40, 35, 30, 25, 20, 15, 10], [float(x) for x in [1.65, 2]]],
            3 : [['char', 'dischar'], ['pulse', 'rest'], [5, 0], [float(x) for x in [1.65, 2]]]}

file_list = []
path_1 = '{}DCPR/'.format(path)
for file_0 in os.listdir(path_1):
    for file_1 in os.listdir(path_1+file_0):
        file_list.append('{}{}/{}'.format(path_1, file_0, file_1))

soh = pd.read_excel('./SoH_expandability.xlsx', sheet_name='SoH', index_col=0)
resistance = pd.read_csv('./DCPR data/INR_21700/20250805_DCPR_resistance.csv', index_col=0)
ir_drop = pd.read_csv('./DCPR data/INR_21700/20250805_DCPR_ir_drop.csv', index_col=0)

cycle = soh.columns
with pd.ExcelWriter('./DCPR data/INR_21700/{}_DCPR_features_21700_inverse.xlsx'.format(dt.datetime.now().strftime('%Y%m%d'))) as writer:
    for e in range(4):
        for a in category[e][2]:
            temp = pd.DataFrame()
            for file in soh.index:
                for cyc in cycle:
                    temp_path='{}{}/'.format(path_1, cyc)
                    if '{}_{}.csv'.format(file, cyc) in os.listdir(temp_path):
                        DCPR = pd.read_csv('{}{}_{}.csv'.format(temp_path, file, cyc))
                        for x, c in enumerate(category[e][3]):
                            for d in category[e][1]:
                                for b in category[e][0]:    
                                    temp_volt = []
                                    for z, y in enumerate(DCPR['{}_{}C_{}_{}'.format(a, c, b, d)]):
                                        temp_volt.append(y)
                                        if (y > 4.2) or (y < 2.5):
                                            if DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+1] == DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+1]:
                                                temp_volt.append(DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+1])
                                            if DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+2] == DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+2]:
                                                temp_volt.append(DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+2])
                                            if DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+3] == DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+3]:
                                                temp_volt.append(DCPR['{}_{}C_{}_{}'.format(a, c, b, d)][z+3])
                                            break
                                        elif y != y:
                                            del temp_volt[-1]
                                            break
                                            
                                    temp.loc['{}_{}'.format(file, cyc), 'SoH'] = soh.loc[file, cyc]
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_{}_kurt'.format(c, b, d)] = pd.DataFrame(temp_volt)[0].kurt()
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_{}_skew'.format(c, b, d)] = pd.DataFrame(temp_volt)[0].skew()
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_{}_initial'.format(c, b, d)] = temp_volt[0]
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_{}_final'.format(c, b, d)] = temp_volt[-1]
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_{}_R'.format(c, b, d)] = resistance.loc[file, '{}_{}_{}C_{}_{}'.format(cyc, a, c, b, d)]
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_{}_IR'.format(c, b, d)] = ir_drop.loc['{}_{}'.format(file, cyc), '{}_{}C_{}_{}'.format(a, c, b, d)]

                                    if b == 'char':
                                        if a >= 10:
                                            temp.loc['{}_{}'.format(file, cyc), '{} C_{}_R_ratio'.format(c, d)] = resistance.loc[file, '{}_{}_{}C_char_{}'.format(cyc, a, c, d)]/resistance.loc[file, '{}_{}_{}C_dischar_{}'.format(cyc, a, c, d)]
                                        else:
                                            temp.loc['{}_{}'.format(file, cyc), '{} C_{}_R_ratio'.format(c, d)] = resistance.loc[file, '{}_{}_{}C_dischar_{}'.format(cyc, a, c, d)]/resistance.loc[file, '{}_{}_{}C_char_{}'.format(cyc, a, c, d)]
            
                                        if d == 'pulse':
                                            temp.loc['{}_{}'.format(file, cyc), '{} C_{}_{}_time'.format(c, b, d)] = len(temp_volt)/10

                                if '50E' in file:
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_hysteresis'.format(c, d)] = (max(DCPR['{}_{}C_char_{}'.format(a, c, d)])-min(DCPR['{}_{}C_dischar_{}'.format(a, c, d)]))/crate_current_50E[str(c)]
                                elif 'M50' in file:
                                    temp.loc['{}_{}'.format(file, cyc), '{} C_{}_hysteresis'.format(c, d)] = (max(DCPR['{}_{}C_char_{}'.format(a, c, d)])-min(DCPR['{}_{}C_dischar_{}'.format(a, c, d)]))/crate_current_M50LT[str(c)]
                            
                            for j in category[e][3][x+1:]:
                                for b in category[e][0]:
                                    # for cyc in cycle:  
                                    if resistance.loc[file, '{}_{}_{}C_{}_{}'.format(cyc, a, c, b, d)] == 0.0:
                                        temp.loc['{}_{}'.format(file, cyc), '{} C_R/{} C_R_{}_{}'.format(j ,c, b, d)] = 0.0
                                    else:
                                        temp.loc['{}_{}'.format(file, cyc), '{} C_R/{} C_R_{}_{}'.format(j ,c, b, d)] = (resistance.loc[file, '{}_{}_{}C_{}_{}'.format(cyc, a, j, b, d)]/resistance.loc[file, '{}_{}_{}C_{}_{}'.format(cyc, a, c, b, d)])

            
            temp.to_excel(writer, sheet_name=str(a))
            print(a, 'done')

In [ ]:
result, accuracy = pd.DataFrame(), pd.DataFrame()
for s in range(0, 100+1, 5):
    df = pd.read_excel('./20250805_DCPR_features_21700_inverse.xlsx', sheet_name=str(s), index_col=0)
    
    if s < 10:
        a, b, c, d, e, f = 1.4, 0.019, 0.15, 0.02, 1.3, 0.0
    elif s < 80:
        a, b, c, d, e, f = 0.9, 0.013, 0.8, 0.011, 1.4, 0.0
    else:
        a, b, c, d, e, f = 0.08, 0.01, 1.0, 0.013, 1.5, 0.002
    
    for i in df.columns:
        if i == '2.0 C_char_rest_IR':
            for j in df.index:
                if '50E' in j:
                    x = 202/166
                elif 'M50' in j:
                    x = 198/166
                    
                df.loc[j, i] = df.loc[j, i]/(a*x)+b
                    
        elif i == '1.65 C_dischar_pulse_IR':
            for j in df.index:
                if '50E' in j:
                    x = 202/166    
                elif 'M50' in j:
                    x = 198/166
                    
                df.loc[j, i] = df.loc[j, i]/(c*x)+d
                    
        elif i == '1.65 C_dischar_pulse_R':
            for j in df.index:
                if '50E' in j:
                    x = 202/166    
                elif 'M50' in j:
                    x = 198/166
                
                df.loc[j, i] = df.loc[j, i]/(e*x)+f
                
    for x, i in enumerate(df.index):
        result.loc[i, s] = ml.predict(pd.DataFrame(mms.transform(df[feature_col]), columns=feature_col))[x]

    for x, i in enumerate(df.index):
        result.loc[i, 'SoH'] = df.loc[i, 'SoH'] 

true, predict = [], []
for i in soc:
    for j in result.index:
        true.append(result.loc[j, 'SoH'])
        predict.append(result.loc[j, i])

rmse = metrics.root_mean_squared_error(true, predict)
mae = metrics.mean_absolute_error(true, predict)
mape = metrics.mean_absolute_percentage_error(true, predict)*100

print('{}\t{}\t{}'.format(round(rmse, 3), round(mae, 3), round(mape, 3)))

plt.plot([70, 105], [70, 105], 'k--')
for s in soc:
    plt.scatter(result['SoH'], result[s], alpha=0.7)
plt.show()